In [1]:
import pandas as pd
import numpy as np
import os
import re
import nltk
from nltk import MWETokenizer, WordNetLemmatizer
from nltk.corpus import wordnet as wn, stopwords
import gensim
from gensim import corpora
from nltk.corpus import reuters

ModuleNotFoundError: No module named 'pyLDAvis'

In [ ]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def normalize(text):
    text = re.sub(r'[^\w\s]',' ',text) # rimuovo la punteggiatura
    text = text.lower()
    text = nltk.pos_tag(text.split()) # prendo i pos tag delle parole (fa anche il tokenizing)
    text = [x for x in text if x[1] in ['NN','NNS','NNP','NNPS']] # mantengo solo i noun
    text = [x[0] for x in text] # rimuovo i pos tag
    text = [lemmatizer.lemmatize(x) for x in text]
    text = [x for x in text if x not in stop_words]
    return text

Il corpus Reuters è un popolare corpus linguistico utilizzato per la ricerca e l'elaborazione del linguaggio naturale. È composto da una vasta collezione di articoli di notizie provenienti dalla Reuters news agency, che copre una vasta gamma di argomenti come politica, economia, finanza, scienza, tecnologia e altro ancora. Il corpus Reuters è spesso utilizzato come benchmark per valutare algoritmi di classificazione del testo, modelli di linguaggio e altre applicazioni nel campo dell'elaborazione del linguaggio naturale. È incluso nella libreria NLTK (Natural Language Toolkit) di Python, che offre un facile accesso a numerosi corpus linguistici.

Contiene 90 argomenti distinti

In [ ]:
ORIGINAL_DOC_IDS = reuters.fileids()
TOPICS = ['trade', 'grain', 'crude', 'acq', 'interest', 'money-fx']

In [ ]:
def get_topics(doc_ids):
    topics = {}
    for doc_id in doc_ids:
        for topic in reuters.categories(doc_id):
            if topic in topics:
                topics[topic] += 1
            else:
                topics[topic] = 1

    #order topics by frequency
    topics = {k: v for k, v in sorted(topics.items(), key=lambda item: item[1], reverse=True)}

    return topics

def get_reuters_docs(test=True):
    docs_ids = reuters.fileids()
    if (test):
        docs_ids = [doc_id for doc_id in ORIGINAL_DOC_IDS if 'test' in doc_id]
        
    i = 0
    length = len(docs_ids)
    while i < length:
        #check if the document has exactly one topic in the list of topics
        if len(set(reuters.categories(docs_ids[i])).intersection(set(TOPICS))) != 1:
            docs_ids.remove(docs_ids[i])
            length -= 1
        else:
            i += 1

    docs = [normalize(reuters.raw(doc_id)) for doc_id in docs_ids]

    return docs, docs_ids

docs, docs_ids = get_reuters_docs()
print('Numero documenti: ' + str(len(docs)))
print('topics: ' + str(TOPICS))
print('\n')

Numero documenti: 1354
topics: ['trade', 'grain', 'crude', 'acq', 'interest', 'money-fx']




In [ ]:
"""
I valori dell'output rappresentano i punteggi di coerenza associati ai rispettivi modelli LDA. 
Questi punteggi sono stati ottenuti mediante l'utilizzo del CoherenceModel. Esso calcola la coerenza 
dei topic utilizzando diverse metriche di coerenza, come ad esempio la metrica C_V. 
Queste metriche valutano la somiglianza semantica tra le parole chiave all'interno di ciascun topic, 
considerando anche la distribuzione delle parole nel corpus di testo.
"""
def fine_tuning(texts, limit, start=2, step=2):
    coherence_values = []
    model_list = []
    num_topic_list = []

    dictionary = corpora.Dictionary(texts)
    dictionary.filter_extremes(no_below=5, no_above=0.3, keep_n=None)  # use Dictionary to remove un-relevant tokens
    corpus = [dictionary.doc2bow(doc) for doc in texts]


    for num_topics in range(start, limit, step):
        print("Elaborazione per k = " + str(num_topics))
        model = gensim.models.LdaModel(corpus, num_topics=num_topics, id2word=dictionary)
        model_list.append(model)
        num_topic_list.append(num_topics)
        coherencemodel = gensim.models.CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, num_topic_list, coherence_values

In [ ]:
model_list, num_topic_list, coherence_values = fine_tuning(docs, 11, start=2, step=2)

Elaborazione per k = 2
Elaborazione per k = 4
Elaborazione per k = 6
Elaborazione per k = 8
Elaborazione per k = 10


In [ ]:
for i in range(0, len(num_topic_list)):
    print(str(num_topic_list[i]) + " topic : " + str(coherence_values[i]))

2 topic : 0.29005203919530526
4 topic : 0.2908663009010457
6 topic : 0.3564919904523756
8 topic : 0.32680786476911294
10 topic : 0.3054230422776756


In [ ]:
for i in range(0, len(num_topic_list)):
    print(f"Modello con {num_topic_list[i]} topic: \n")
    for topic_id, topic in model_list[i].show_topics(formatted=True, num_topics=num_topic_list[i], num_words=10):
        print(f"Topic {topic_id}: {topic}")
    print('\n')

Modello con 2 topic: 

Topic 0: 0.019*"bank" + 0.016*"rate" + 0.013*"market" + 0.012*"trade" + 0.012*"year" + 0.011*"share" + 0.011*"oil" + 0.010*"group" + 0.009*"stock" + 0.009*"dollar"
Topic 1: 0.031*"share" + 0.016*"oil" + 0.015*"price" + 0.014*"stock" + 0.013*"year" + 0.012*"offer" + 0.009*"market" + 0.009*"corp" + 0.009*"inc" + 0.008*"group"


Modello con 4 topic: 

Topic 0: 0.031*"share" + 0.016*"rate" + 0.015*"year" + 0.014*"stock" + 0.013*"bank" + 0.012*"oil" + 0.010*"offer" + 0.009*"corp" + 0.009*"day" + 0.008*"group"
Topic 1: 0.020*"price" + 0.017*"oil" + 0.015*"tonne" + 0.015*"rate" + 0.013*"corp" + 0.011*"year" + 0.011*"market" + 0.009*"share" + 0.009*"stock" + 0.009*"official"
Topic 2: 0.023*"share" + 0.016*"stock" + 0.014*"market" + 0.013*"trade" + 0.013*"group" + 0.013*"inc" + 0.012*"tonne" + 0.012*"sale" + 0.012*"year" + 0.010*"offer"
Topic 3: 0.017*"bank" + 0.017*"oil" + 0.015*"dollar" + 0.014*"market" + 0.014*"share" + 0.012*"price" + 0.011*"trade" + 0.011*"group" + 0